Install the required libraries

In [ ]:
!pip3 install torch torchvision torchaudio 
!pip3 install transformers
!pip3 install sklearn
!pip3 install pandas
!pip3 install numpy
!pip3 install tabulate
!pip3 install tqdm

Load the required libraries

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tabulate import tabulate
from tqdm import trange
import random
from sklearn.metrics import accuracy_score, f1_score
import time

Code for RoBERTa Classifier

In [ ]:
# Load the dataset
data = pd.read_csv('IMDB_Reviews_Top_250_preprocessed_without_stopwords.csv')

In [ ]:
data.head(5)

In [ ]:
# Set the threshold for binary conversion
threshold = 7.0

# Convert the 'Rating' column to a binary variable
data['Binary Rating'] = data['Rating'].apply(lambda x: 1 if x >= threshold else 0)

In [ ]:
# Extract reviews texts and ratings into arrays
reviews = data['Review Text'].values
labels = data['Binary Rating'].values

In [ ]:
# Shouldn't be necessary but just to be safe
for review in reviews:
    str(review)  

In [ ]:
# Count the number of rows with label 1
num_label_1 = data[data['Binary Rating'] == 1]['Binary Rating'].count()

# Count the number of rows with label 0
num_label_0 = data[data['Binary Rating'] == 0]['Binary Rating'].count()

print(f"Number of rows with Binary Rating 1: {num_label_1}")
print(f"Number of rows with Binary Rating 0: {num_label_0}")

In [ ]:
reviews[0:5]

In [ ]:
labels[0:5]

In [ ]:
# Load RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
# testing tokens and token Ids for a random sentence
def print_rand_sentence():
    index = random.randint(0, len(reviews)-1) #random index in texts list
    table = np.array([tokenizer.tokenize(reviews[index]), 
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(reviews[index]))]).T #tokenize random text in texts list
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))  #print in table format

print_rand_sentence()

In [ ]:
# Find max sequensce lenght

MAX_LEN = 0
for review in reviews:
    tokenized = tokenizer(review,return_tensors='pt',add_special_tokens=True)
    MAX_LEN = max(MAX_LEN, tokenized['input_ids'].size()[1])

In [ ]:
MAX_LEN

In [ ]:
# Tokenize and encode texts then extract token ids, attention masks and labels in torch tensor format.
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
    return tokenizer.encode_plus(                    #returns dictionary with token ids, attention masks and token type ids
                        input_text,
                        max_length = 512,            
                        padding = 'max_length',    #padding tokens to be of the same size
                        truncation=True,           # Truncate the text if it exceeds 512 tokens
                        return_attention_mask = True,
                        return_tensors = 'pt'        #torch tensor format
                   )


for sample in reviews:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids']) 
    attention_masks.append(encoding_dict['attention_mask'])

#concatinating the tesnors in a single dimension
token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

In [ ]:
token_id

In [ ]:
attention_masks

In [ ]:
labels

In [ ]:
def print_rand_sentence_encoding():
    index = random.randint(0, len(reviews) - 1)
    tokens = tokenizer.convert_ids_to_tokens(token_id[index].numpy().tolist())  # Convert token_ids to a list of strings
    token_ids = token_id[index].numpy()
    attention = attention_masks[index].numpy()

    table = np.array([tokens, token_ids, attention]).T
    print(tabulate(table, headers=['Tokens', 'Token IDs', 'Attention Mask'], tablefmt='fancy_grid'))

print_rand_sentence_encoding()

In [ ]:
# split data in training and validation sets

val_ratio = 0.2 
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# DataLoaders
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [ ]:
# load RoBERTa model as a RobertaForSequenceClassification model
model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
)

optimizer = torch.optim.AdamW(
                model.parameters(),
                lr=3e-5,     # You can also try 3e-5, 2e-5
                eps=1e-8     # AdamW's epsilon value. Probably optimal.
                )

In [ ]:
# Define device and move the model to it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
epochs = 4
debug_interval = 60  # Print a debug message every 60 seconds

# Define variables for early stopping
patience = 3
min_delta = 0.001
best_val_loss = float('inf')
counter = 0

start_time = time.time()  # Record the start time

for epoch in trange(epochs, desc='Epoch'):
    # Set model to training mode for training loop
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

        # Check if five seconds have passed, and print a debug message
        if time.time() - start_time > debug_interval:
            print(f"Epoch {epoch + 1}/{epochs}, Step {step + 1}/{len(train_dataloader)}, "
                  f"Train Loss: {tr_loss / nb_tr_steps:.4f}")
            start_time = time.time()  # Reset the start time for the next 60 seconds

    # After the training loop, save the model checkpoint for each epoch
    torch.save(model.state_dict(), f'model_checkpoint_epoch_RoBERTa_Classifier_{epoch + 1}.pt')
    
    # Set model to evaluation mode for validation loop
    model.eval()
    
    # Initialise metrics 
    val_loss = 0
    val_preds = []
    val_labels = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            # Forward pass
            eval_output = model(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_input_mask, 
                              labels=b_labels)
        logits = eval_output.logits
        eval_loss = eval_output.loss
        val_loss += eval_loss.item()

        # Calculate validation metrics
        preds = torch.argmax(logits, dim=1).cpu().detach().numpy()
        label_ids = b_labels.cpu().detach().numpy()
        val_preds.extend(preds)
        val_labels.extend(label_ids)

    # Calculate average metrics over all batches
    avg_val_loss = val_loss / len(validation_dataloader)
    avg_val_accuracy = accuracy_score(val_labels, val_preds)
    avg_val_f1 = f1_score(val_labels, val_preds, average='weighted')

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\n\t - Validation loss: {:.4f}'.format(avg_val_loss))
    print('\n\t - Validation accuracy: {:.4f}'.format(avg_val_accuracy))
    print('\n\t - Validation F1 score: {:.4f}'.format(avg_val_f1))

    # Check if validation loss improved
    if avg_val_loss < best_val_loss - min_delta * best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
    else:
        counter += 1
        if counter == patience:
            print("Validation loss did not improve for {} epochs. Early stopping...".format(patience))
            break

# After the training loop finishes, close the results file
results_file.close()